In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

In [ ]:
# define various hyperparameters
nb_train_samples = 5622
nb_validation_samples = 750
batch_size = 16
epochs = 50

In [ ]:
# load the pre-trained model for transfer learning
img_width, img_height = 256, 256
model = applications.VGG19(weights="imagenet", include_top=False, input_shape=(
    img_width, img_height, 3))

# to load other models follow use the following parameters:
"""
{ 
    "xception":    { "width": 299, "height": 299 },
    "vgg16":       { "width": 224, "height": 224 },
    "vgg19":       { "width": 224, "height": 224 },
    "resnet50":    { "width": 224, "height": 224 },
    "inceptionv3": { "width": 299, "height": 299 },
    "mobilenet":   { "width": 224, "height": 224 }
}
"""

In [ ]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False

In [ ]:
# adding custom Layers
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(3, activation="softmax")(x)

In [ ]:
# creating the final model
model_final = Model(input=model.input, output=predictions)

# compile the model
model_final.compile(loss="categorical_crossentropy",
                    optimizer=optimizers.SGD(lr=0.0001, momentum=0.9),
                    metrics=["accuracy"]
                    )

In [ ]:
# Initiate the train and test generators with data Augumentation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    horizontal_flip=False)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    'train_set_paintings//',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical"
)

validation_generator = test_datagen.flow_from_directory(
    'validation_set_paintings//',
    target_size=(img_height, img_width),
    class_mode="categorical"
)

In [ ]:
# Save the model according to the conditions
checkpoint = ModelCheckpoint("vgg19_1.h5", monitor='val_acc',
                             verbose=1, save_best_only=True,
                             save_weights_only=False,
                             mode='auto', period=1)
# monitor the loss
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [ ]:
# Train the model
hist = model_final.fit_generator(
    train_generator,
    steps_per_epoch=1800 // batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=250 // batch_size,
    callbacks=[checkpoint, early],
    workers=8  # cpu generation is run in parallel to the gpu training
)

print("Maximum train accuracy:", max(hist.history["acc"]))
print("Maximum train accuracy on epoch:", hist.history["acc"].index(max(hist.history["acc"])) + 1)

print("Maximum validation accuracy:", max(hist.history["val_acc"]))
print("Maximum validation accuracy on epoch:", hist.history["val_acc"].index(max(hist.history["val_acc"])) + 1)

"""
Epoch 00051: early stopping
Maximum train accuracy: 0.955357142857
Maximum train accuracy on epoch: 52
Maximum validation accuracy: 0.75757575835
Maximum validation accuracy on epoch: 41
"""

In [ ]:
# visualize the results
import matplotlib.pyplot as plt


def plot_history(hist):
    plt.figure()
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.legend(['Training', 'Validation'])
    plt.savefig('paintings_loss_vgg19.png', dpi=400)

    plt.figure()
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.plot(hist.history['acc'])
    plt.plot(hist.history['val_acc'])
    plt.legend(['Training', 'Validation'])
    plt.savefig('paintings_accuracy_vgg19.png', dpi=400)


plot_history(hist)